In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

from ibm_botocore.client import Config
import ibm_boto3
import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F


import numpy as np
import pandas as pd
import json
import pyspark.sql
from pyspark.sql.functions import col, skewness, kurtosis
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import when
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from datetime import date, timedelta, datetime
import time

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210210214214-0000
KERNEL_ID = 42653e5d-008d-46e7-adec-ead05cdc06bc


Hidden cells with credentials for accessing parquets from Cloud Object Storage

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
#cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

#movies_df = spark.read.parquet(cos.url('moviesdataframe.parquet', <BUCKET-NAME>))
movies_df.createOrReplaceTempView("movies")

In [4]:
result = spark.sql("""
            Select year as Year,COUNT(year) as Number_of_Movies
            FROM movies 
            GROUP BY year
            ORDER BY year ASC
                   """).repartition(1)

result.show()


+----+----------------+
|Year|Number_of_Movies|
+----+----------------+
|1919|               3|
|1920|               2|
|1921|               1|
|1922|               2|
|1923|               3|
|1925|               6|
|1926|               8|
|1927|               6|
|1928|               3|
|1929|               3|
|1930|               7|
|1931|               7|
|1932|               7|
|1933|               7|
|1934|               7|
|1935|               6|
|1936|               8|
|1937|              11|
|1938|               6|
|1939|              11|
+----+----------------+
only showing top 20 rows



In [5]:
## To save it locally
from IPython.display import HTML 
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "result.csv"):  
    csv = df.to_csv(sep=',',header=True, index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# sparkDF to Pandas
result_df = result.toPandas()
create_download_link(result_df,filename='movies_each_year.csv')